# RAG

In [ ]:
!pip install --upgrade --quiet langchain langchain-openai faiss-cpu tiktoken

In [1]:
from operator import itemgetter

from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

In [2]:
vectorstore = FAISS.from_texts(
    ["피터는 구글에서 일합니다", "핸리는 켄쇼에서 일합니다", "로버트는 메타에서 일합니다."], embedding=OpenAIEmbeddings()
)
retriever = vectorstore.as_retriever()

In [3]:
retriever.invoke("로버트는 어디서 일해?")

[Document(id='bf25889b-7f82-47d1-9f61-15ece2867b88', metadata={}, page_content='로버트는 메타에서 일합니다.'),
 Document(id='351694df-18e4-4287-a633-5809f8bfdf9b', metadata={}, page_content='피터는 구글에서 일합니다'),
 Document(id='6f62834e-59bb-41a1-8254-94855a7a07e5', metadata={}, page_content='핸리는 켄쇼에서 일합니다')]

In [4]:
template = """다음 문맥에만 근거하여 문제에 답하세요:
{context}

질문: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

model = ChatOpenAI()

In [5]:
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

In [6]:
chain.invoke("로버트는 어디서 일해?")

'로버트는 메타에서 일합니다.'

In [7]:
template = """다음 문맥에만 근거하여 문제에 답하세요:
{context}

질문: {question}

답은 다음의 언어로 대답하세요.: {language}
"""
prompt = ChatPromptTemplate.from_template(template)

chain = (
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question"),
        "language": itemgetter("language"),
    }
    | prompt
    | model
    | StrOutputParser()
)

In [8]:
chain.invoke({"question": "피터는 어디서 일해?", "language": "english"})

'Peter works at Google.'